In [1]:
from fbprophet import Prophet
import pandas as pd
import numpy as np
import plotly
import gc
import os
os.environ['NUMEXPR_MAX_THREADS'] = '8'

In [2]:
data = pd.read_pickle('./data/dat.pkl')

In [3]:
train = data[data.date_block_num <= 33][['date', 'item_cnt_month', 'item_price']]
train['revenue'] = np.exp(train['item_cnt_month'] - 1) * np.exp(train['item_price'])
train = train[['date', 'revenue']]

train.columns = ['ds', 'y']

train = train.groupby('ds').agg({'y':'sum'})
train.reset_index(inplace=True)
train['y'] = np.log(train['y'])

del data
gc.collect();

In [4]:
model = Prophet(growth='linear', daily_seasonality=True)
model.fit(train)

In [5]:
future = model.make_future_dataframe(periods = 30)
pred = model.predict(future)

In [6]:
test_pred = pred[['yhat_upper', 'yhat', 'yhat_lower']].iloc[-30:]
test_pred.columns = ['prophet_yhat_upper', 'prophet_yhat', 'prophet_yhat_lower']
test_pred.to_csv("./features/feature_prophet.csv", index=False)